In [2]:
import pandas as pd
import numpy as np

In [3]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data

--2025-10-13 22:28:10--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.003s  

2025-10-13 22:28:10 (29.6 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [192]:
df = pd.read_csv('course_lead_scoring.csv')

# Getting Size
df.shape
df = df.drop('lead_score', axis=1)

In [193]:
# Looking at the head
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,1
1,social_media,retail,1,46992.0,employed,south_america,1,0
2,events,healthcare,5,78796.0,unemployed,australia,3,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0
4,referral,education,3,85012.0,self_employed,europe,3,1


In [194]:
# Transpose of Head
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
converted,1,0,1,0,1


In [195]:
# Types inspection
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
converted                     int64
dtype: object

In [196]:
# getting categorical columns
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [197]:
# prepare uniform values in column names (not needed but done for practice)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# preparing uniform values in column values
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')
 
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
converted,1,0,1,0,1


In [198]:
# filling NAs of categorical_columns
df[categorical_columns] = df[categorical_columns].fillna('NA')
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NA,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NA,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
converted,1,0,1,0,1


In [199]:
# getting numerical columns
numerical_columns = ['number_of_courses_viewed','annual_income','interaction_count']
numerical_columns

['number_of_courses_viewed', 'annual_income', 'interaction_count']

In [200]:
# filling NAs of numerical columns
df[numerical_columns] = df[numerical_columns].fillna(0.0)
df[numerical_columns]

,number_of_courses_viewed,annual_income,interaction_count
0,1,79450.0,4
1,1,46992.0,1
2,5,78796.0,3
3,2,83843.0,1
4,3,85012.0,3
...,...,...,...
1457,1,0.0,4
1458,3,65259.0,2
1459,1,45688.0,3
1460,5,71016.0,0


In [202]:
# Question 3
from sklearn.model_selection import train_test_split

In [203]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
len(df_full_train), len(df_test)

(1169, 293)

In [204]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val)

(876, 293)

In [205]:
# Reset Index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [206]:
# Prepare y
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [207]:
df_full_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0
638,events,retail,3,38048.0,student,north_america,6,1
799,social_media,education,2,71399.0,NA,europe,1,1
380,referral,education,2,47912.0,employed,australia,1,0
303,paid_ads,healthcare,1,34806.0,employed,europe,4,1
...,...,...,...,...,...,...,...,...
1130,events,manufacturing,1,57039.0,employed,south_america,2,0
1294,events,healthcare,2,56185.0,student,south_america,2,0
860,paid_ads,manufacturing,1,56402.0,student,north_america,1,0
1459,paid_ads,technology,1,45688.0,student,north_america,3,1


In [208]:
# Calculate mutual info
from sklearn.metrics import mutual_info_score

def mutual_info_converted_score(series):
    return mutual_info_score(series, df_train.converted)

In [209]:
mi = df_train[categorical_columns].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

In [210]:
# Question 4 - Train Model

In [211]:
# Delete column converted
del df_train['converted']
del df_val['converted']
del df_test['converted'] 

In [212]:
# One-hot Encoding
from sklearn.feature_extraction import DictVectorizer

In [213]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [214]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(xi):
    score = w0
     
    for j in range(len(w)):
        score = score + xi[j] * w[j]
     
    result = sigmoid(score)
    return result

In [215]:
from sklearn.linear_model import LogisticRegression
 
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [216]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.05 ,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.313,  0.02 ,
       -0.012, -0.012, -0.116,  0.08 , -0.03 ,  0.004, -0.011, -0.011,
       -0.006,  0.008,  0.006, -0.033, -0.025,  0.456])

In [217]:
# bias
model.intercept_[0]

np.float64(-0.06891397780582399)

In [218]:
model.predict(X_train)

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [219]:
y_pred = model.predict_proba(X_val)[:,1]
y_pred

array([0.61641021, 0.79924378, 0.53410391, 0.47095791, 0.57411527,
       0.44921366, 0.87163964, 0.84797997, 0.83526881, 0.62055739,
       0.5520865 , 0.77959866, 0.68786689, 0.77535978, 0.53429569,
       0.91867361, 0.5343365 , 0.4180206 , 0.29540696, 0.85120469,
       0.79475779, 0.73744094, 0.44405776, 0.6411325 , 0.41342195,
       0.75727078, 0.90212882, 0.34486756, 0.42572004, 0.96746988,
       0.9204702 , 0.37261045, 0.65543887, 0.90494541, 0.75385421,
       0.64095535, 0.82117516, 0.834098  , 0.6594366 , 0.31221599,
       0.78828955, 0.3495941 , 0.96469813, 0.62700979, 0.50688783,
       0.53760041, 0.82487922, 0.73757621, 0.73483226, 0.69500698,
       0.47570811, 0.84993978, 0.56097755, 0.92763697, 0.65148536,
       0.62136979, 0.63984995, 0.2874928 , 0.47392408, 0.57903449,
       0.35047775, 0.62321169, 0.39211954, 0.61461065, 0.8525382 ,
       0.75293349, 0.89017704, 0.72487348, 0.95514052, 0.89109422,
       0.75162671, 0.33788115, 0.60727014, 0.52075568, 0.63572

In [220]:
converted_decision = y_pred >0.5
converted_decision

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,

In [221]:
df_val[converted_decision]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count
0,paid_ads,healthcare,3,52220.0,unemployed,europe,1
1,organic_search,technology,3,59656.0,unemployed,middle_east,4
2,events,manufacturing,0,57134.0,self_employed,north_america,4
4,referral,retail,1,54103.0,unemployed,south_america,3
6,organic_search,NA,1,0.0,student,asia,5
...,...,...,...,...,...,...,...
285,referral,education,3,0.0,unemployed,asia,0
286,events,technology,1,0.0,employed,asia,2
287,events,retail,5,69305.0,unemployed,australia,3
289,paid_ads,NA,1,20326.0,employed,middle_east,3


In [222]:
# Accuracy
(y_val == converted_decision).mean()

np.float64(0.7064846416382252)